In [1]:
import math
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import numpy as np
import os
import re
import seaborn as sns

from landscape import Landscape
from util import Type

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [11]:
def generateRoadAngleGraph(folderpath):        
    for _, _, filenames in os.walk(folderpath):
        filenames = [f for f in filenames if f.endswith(".p")]

        list_of_degrees_mi = []
        list_of_degrees_ma = [] 
        
        for f in filenames:
            f = f[:f.index(".p")]
#             print ("Filename = {}.".format(f))

            filename_state = "{}/{}.p".format(folderpath, f)
            filename_roads = "{}/{}.output.txt".format(folderpath, f)
            filename_stats = "{}/stats_{}.output.txt".format(folderpath, f)

            landscape = Landscape(100, 100, None, 0, 0, 0, 0, load_filename=filename_state) #hard-code size=100
            for rs in landscape.roadsegments:
                (x, y) = (rs.rnode2.x - rs.rnode1.x, rs.rnode2.y - rs.rnode1.y)
                r = math.hypot(x, y)
                theta = np.arctan2(y, x)
                theta = math.degrees(theta) % 360
                if Type.MAJOR_ROAD in rs.rnode1.type:
                    list_of_degrees_ma.append((r, theta))
                else:
                    list_of_degrees_mi.append((r, theta))

        list_of_degrees_ma = sorted(list_of_degrees_ma, key=lambda x: x[1])
        list_of_degrees_mi = sorted(list_of_degrees_mi, key=lambda x: x[1])
        directions_ma = set([theta for (r, theta) in list_of_degrees_ma])
        directions_mi = set([theta for (r, theta) in list_of_degrees_mi])
        summed_list_of_degrees_ma = []
        summed_list_of_degrees_mi = []

        for direction in directions_ma:
            dirList = [r for (r, theta) in list_of_degrees_ma if theta == direction]
            magnitude = sum(dirList)/len(filenames)
            summed_list_of_degrees_ma.append((magnitude, direction))
        for direction in directions_mi:
            dirList = [r for (r, theta) in list_of_degrees_mi if theta == direction]
            magnitude = sum(dirList)/len(filenames)
            summed_list_of_degrees_mi.append((magnitude, direction))
        
        width, height = matplotlib.rcParams['figure.figsize']
        size = min(width, height)

        print (len(filenames))
        
        fig = plt.figure(figsize=(size, size))
        ax = fig.add_axes([0, 0, 1, 1], polar=True)
        ax.set_rlim(0, 4)
        
        for r, theta in summed_list_of_degrees_ma:
            ax.plot([math.radians(theta), math.radians(theta)], [0, r], color='#226622')
        ax.grid(False)
        ax.spines['polar'].set_visible(False)
        ax.set_xticklabels([])
        plt.show()
        
        fig = plt.figure(figsize=(size, size))
        ax = fig.add_axes([0, 0, 1, 1], polar=True)
        ax.set_rlim(0, 50)
        
        for r, theta in summed_list_of_degrees_mi:
            ax.plot([math.radians(theta), math.radians(theta)], [0, r], color='#222266')
        ax.grid(False)
        ax.spines['polar'].set_visible(False)
        ax.set_xticklabels([])
        plt.show()

In [19]:
def generateConnectivityGraph(folderpath):
    for _, _, filenames in os.walk(folderpath):
        filenames = [f for f in filenames if f.endswith(".p")]
        
        connectivity = []
        local_depth = []
        global_depth = []

        for f in filenames:
            f = f[:f.index(".p")]
#             print ("Filename = {}.".format(f))

            filename_state = "{}/{}.p".format(folderpath, f)
            filename_roads = "{}/{}.output.txt".format(folderpath, f)
            filename_stats = "{}/stats_{}.output.txt".format(folderpath, f)
            
            # skip first two numbers x and y, match next 3 numbers
            p = re.compile("\D+\d+\D+\d+\D+(\d+)\D+(\d+)\D+(\d+)\D+") 
            
            with open(filename_stats, "rb") as file:
                line = file.readline()
                c = []
                l = []
                g = []
                while line:
                    m = p.match(str(line))
                    connectivity.append(int(m.groups()[0]))
                    local_depth.append(int(m.groups()[1]))
                    global_depth.append(int(m.groups()[2]))
                    line = file.readline()
        
        data = local_depth
        fig1, ax1 = plt.subplots()

        ax1.hist([data], int(max(data)), alpha=0.75, orientation='horizontal', log=True)
        ax1.set_ylim(0, 40)
        plt.show()
    
        data = global_depth
        fig1, ax1 = plt.subplots()
        ax1.hist([data], 90, alpha=0.75, orientation='horizontal', log=True)
        ax1.set_ylim(0, 400)
        plt.show()


In [2]:
import random
import copy 
from util import *
def generateCurvinessGraph(folderpath):
    for _, _, filenames in os.walk(folderpath):
        filenames = [f for f in filenames if f.endswith(".p")]

        curvature_list = []
        for f in filenames:
            f = f[:f.index(".p")]
#             print ("Filename = {}.".format(f))

            filename_state = "{}/{}.p".format(folderpath, f)
            filename_roads = "{}/{}.output.txt".format(folderpath, f)
            filename_stats = "{}/stats_{}.output.txt".format(folderpath, f)

            landscape = Landscape(100, 100, None, 0, 0, 0, 0, load_filename=filename_state) #hard-code size=100
            landscape.set_adjacents()
#             array = np.ones((len(landscape.array), len(landscape.array[0])))
#             array *= -1


#             for i in range(100):
#                 for j in range(100):
#                     if landscape.array[i][j] in landscape.roadnodes:
#                         array[i][j] = 0

            list_of_roads = []
            roadnodes = set(copy.copy(landscape.roadnodes))
            for n in sorted(landscape.roadnodes, key=lambda x: (x.x, x.y)):
                if n not in roadnodes:
                    continue
                new_road = set([n])
                new_road_bits = [x for nn in new_road for x in nn.adjacent if len(nn.adjacent & set(landscape.roadnodes)) <= 2 and x in landscape.roadnodes and x not in new_road]
                while len(new_road_bits) > 0:
                    new_road.update(set(new_road_bits))
                    new_road_bits = [x for nn in new_road_bits for x in nn.adjacent if len(nn.adjacent & set(landscape.roadnodes)) <= 2 and x in landscape.roadnodes and x not in new_road]
                list_of_roads.append(new_road)
                roadnodes -= set(new_road)

            for road in list_of_roads:
                road = list(road)
                if len(road) > 2:
                    # sort roadnodes of road according to x val
                    road = sorted(road, key=lambda x: (x.x, x.y))
                    x = np.array([n.x for n in road])
                    y = np.array([n.y for n in road])
                    dx_dt = np.gradient(x)
                    dy_dt = np.gradient(y)
                    d2x_dt2 = np.gradient(dx_dt)
                    d2y_dt2 = np.gradient(dy_dt)
                    curvature = np.abs(d2x_dt2 * dy_dt - dx_dt * d2y_dt2) / (dx_dt * dx_dt + dy_dt * dy_dt)**1.5
                    curvature_list.append((sum(curvature), len(road)))

        curvatures = set([x[0] for x in curvature_list])
        new_curvature_list = []
        for c in curvatures:
            new_curvature_list.append((c, sum([x[1] for x in curvature_list if x[0] == c])))
        maxcolor = 3.9908035449766794
        new_curvature_list = sorted(new_curvature_list, key=lambda x:x[0])
        cmap = matplotlib.cm.get_cmap('nipy_spectral')

        fig, ax = plt.subplots(figsize=(1, 5))
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.axis('off')

#             for road in list_of_roads:
#                 road = list(road)
#                 if len(road) > 2:
#                     # sort roadnodes of road according to x val
#                     road = sorted(road, key=lambda x: (x.x, x.y))
#                     x = np.array([n.x for n in road])
#                     y = np.array([n.y for n in road])
#                     dx_dt = np.gradient(x)
#                     dy_dt = np.gradient(y)
#                     d2x_dt2 = np.gradient(dx_dt)
#                     d2y_dt2 = np.gradient(dy_dt)
#                     curvature = np.abs(d2x_dt2 * dy_dt - dx_dt * d2y_dt2) / (dx_dt * dx_dt + dy_dt * dy_dt)**1.5
#                     for n in road:
#                         array[n.x][n.y] = sum(curvature)/maxcolor

        curr=None
        for c in new_curvature_list:
            if curr is None:
                curr = [0]
            color = ((c[0]/maxcolor) + 0.1) / 1.2
            color = cmap(color)

            plt.bar([0], [c[1]], color=color, bottom=curr)
            curr = [curr[0] + c[1]]

#             plt.imshow(array, cmap=plt.cm.nipy_spectral, vmin=-0.1, vmax=1.1)